# Sharing my Best rules bot

**This bot is a fork of my best rules based bot - it is currently in 71st place at 1020.7  as of 10/17, but has been as high as 1055 so far and has beat some 1100 bots.**

With the new week, I am moving on to "trying" to build an RL bot with PyTorch - I did a simple RL project years ago, but I am just getting going on Deep RL and PyTorch.  I have spent my time this last week working on understanding the environment and working on a rules based bot to get a feel for how much success a rules based bot can have. 

This competition brings back memories of my Artificial Intelligence for Robotics at Georgia Tech... that was an amazing class that was hard, but I loved every minute.

I want to give credit to @eugenkeil (notebook baseline bot) and @raffaelemorganti (notebook rule-based way) that really helped me hit the ground running and gave me many many ideas for making a good rule based bot.

The major items in this bot that I contirbuted are a VERY CRUDE system (run_pass function) that determines where defenders are and tries to run around them or pass to open teammates nearby.  The system also runs to where the ball is going when it doesn't have the ball and then moves to ball when close - this helps the bots defense IMO and makes a big difference.

I hope it helps and if so, please share ideas and code with others as that is how we all learn.  Also, if you find this useful and have an code you'd be willing to share to help me get a jumpstart on training my Deep RL with Pytorch I will gladly accept it.

In [ ]:
# Install:
# Kaggle environments.
!git clone https://github.com/Kaggle/kaggle-environments.git
!cd kaggle-environments && pip install .

# GFootball environment.
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.6 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.6.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

In [ ]:
%%writefile submission.py
from kaggle_environments.envs.football.helpers import *
from math import sqrt

directions = [
[Action.TopLeft, Action.Top, Action.TopRight],
[Action.Left, Action.Idle, Action.Right],
[Action.BottomLeft, Action.Bottom, Action.BottomRight]]

dirsign = lambda x: 1 if abs(x) < 0.01 else (0 if x < 0 else 2)

enemyGoal = [1, 0]
perfectRange = [[0.61, 1], [-0.2, 0.2]]

def inside(pos, area):
    return area[0][0] <= pos[0] <= area[0][1] and area[1][0] <= pos[1] <= area[1][1]

def get_distance(pos1,pos2):
    return ((pos1[0]-pos2[0])**2+(pos1[1]-pos2[1])**2)**0.5

def player_direction(obs):
    controlled_player_pos = obs['left_team'][obs['active']]
    controlled_player_dir = obs['left_team_direction'][obs['active']]
    x = controlled_player_pos[0]
    y = controlled_player_pos[1]
    dx = controlled_player_dir[0]
    dy = controlled_player_dir[1]
    
    if x <= dx:
        return 0
    if x > dx:
        return 1

def run_pass(left_team,right_team,x,y):
    ###Are there defenders dead ahead?
    defenders=0
    for i in range(len(right_team)):
        if right_team[i][0] > x and y +.01 >= right_team[i][1] and right_team[i][1]>= y - .01:
            if abs(right_team[i][0] - x) <.01:
                defenders=defenders+1
    if defenders == 0:
        return Action.Right
    
    teammateL=0
    teammateR=0
    for i in range(len(left_team)):
        #is there a teamate close to left
        if left_team[i][0] >= x:
            if left_team[i][1] < y:
                if abs(left_team[i][1] - x) <.05:
                    teammateL=teammateL+1
        
        #is there a teamate to right
        if left_team[i][0] >= x:
            if left_team[i][1] > y:
                if abs(left_team[i][1] - x) <.05:
                    teammateR=teammateR+1
    #pass only close to goal
    if x >.75:
        if teammateL > 0 or teammateR > 0:
            return Action.ShortPass
    
    if defenders > 0 and y>=0:
        return Action.TopRight
    
    if defenders > 0 and y<0:
        return Action.BottomRight

@human_readable_agent
def agent(obs):
    controlled_player_pos = obs['left_team'][obs['active']]
    
    # special plays
    if obs["game_mode"] == GameMode.Penalty:
        return Action.Shot
    if obs["game_mode"] == GameMode.Corner:
        if controlled_player_pos[0] > 0:
            return Action.Shot
    if obs["game_mode"] == GameMode.FreeKick:
        return Action.Shot
    
    # Make sure player is running.
    if  0 < controlled_player_pos[0] < 0.6 and Action.Sprint not in obs['sticky_actions']:
        return Action.Sprint
    elif 0.6 < controlled_player_pos[0] and Action.Sprint in obs['sticky_actions']:
        return Action.ReleaseSprint

    # Does the player we control have the ball?
    if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:
        
        goalkeeper = 0
        #if in the zone near goal shoot
        if inside(controlled_player_pos, perfectRange) and controlled_player_pos[0] < obs['ball'][0]:
            return Action.Shot
        #if the goalie is coming out on player near goal shoot
        elif abs(obs['right_team'][goalkeeper][0] - 1) > 0.2 and controlled_player_pos[0] > 0.4 and abs(controlled_player_pos[1]) < 0.2:
            return Action.Shot
        # if close to goal and too wide for shot pass the ball
        if controlled_player_pos[0] >.75 and controlled_player_pos[1] >.20 or controlled_player_pos[0] >.75 and controlled_player_pos[1] <-.20 :
            return Action.ShortPass
        # if near our goal and moving away long pass to get out of our zone
        if player_direction(obs)==1 and controlled_player_pos[0]<-.3:
            return Action.LongPass
        # which way should we run or pass
        else:
            return run_pass(obs['left_team'],obs['right_team'],controlled_player_pos[0],controlled_player_pos[1])
    else:
        #vector where ball is going
        ball_targetx=obs['ball'][0]+obs['ball_direction'][0]
        ball_targety=obs['ball'][1]+obs['ball_direction'][1]
        
        #euclidian distance to the ball so we head off movement until very close
        e_dist=get_distance(obs['left_team'][obs['active']],obs['ball'])
        
        #if not close to ball move to where it is going
        if e_dist >.005:
            # Run where ball will be
            xdir = dirsign(ball_targetx - controlled_player_pos[0])
            ydir = dirsign(ball_targety - controlled_player_pos[1])
            return directions[ydir][xdir]
        #if close to ball go to ball
        else:
            # Run towards the ball.
            xdir = dirsign(obs['ball'][0] - controlled_player_pos[0])
            ydir = dirsign(obs['ball'][1] - controlled_player_pos[1])
            return directions[ydir][xdir]

In [ ]:
# Set up the Environment.
from kaggle_environments import make
env = make("football", configuration={"save_video": True, "scenario_name": "11_vs_11_kaggle", "running_in_notebook": True})
output = env.run(["/kaggle/working/submission.py", "do_nothing"])[-1]
print('Left player: reward = %s, status = %s, info = %s' % (output[0]['reward'], output[0]['status'], output[0]['info']))
print('Right player: reward = %s, status = %s, info = %s' % (output[1]['reward'], output[1]['status'], output[1]['info']))
env.render(mode="human", width=800, height=600)